<a href="https://colab.research.google.com/github/KevinLolochum/A-B-Testing-a-landing-page/blob/main/A_B_testing_a_landing_page.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

The data contains 300 participants who are assigned to the control group and the treatment group at random. The control group is shown the original version of the landing page while the treatment group is shown the redisigned version. The goal is to determine whether the new page makes site visitors stay in the site for longer.

In [1]:
# Loading the data
import pandas as pd
from google.colab import files
uploaded = files.upload()


Saving abtest.csv to abtest.csv


In [2]:
df = pd.read_csv('abtest.csv')
df.head()

,group,time_on_page,browser,click
0,Control,1.748084,Safari,False
1,Treatment,5.947352,Firefox,True
2,Control,2.007091,Chrome,False
3,Treatment,1.633244,Safari,True
4,Control,0.560507,Chrome,True


Testing inclusion criteria to ensure there is no bias between observed variables.

The observed variable here is browser.

In [3]:
import numpy as np

# I will start by creating a contigency table using pd.pivot or pd.crosstab
# Piv = df.pivot_table('click', index = 'browser', columns= 'group', aggfunc= np.sum)
Cont = pd.crosstab(df['browser'], df['group'])
Cont


group,Control,Treatment
browser,,
Chrome,73,71
Edge,17,19
Firefox,26,34
Safari,26,34


From eyeballing, there are more subjects in the control group for chrome viewers as opposed to firefox where there are more viewers assigned to the treatment group.

This shows that assignment is random and no one group was assigned more viewers across the board.

This can be tested using chi_squared test.

Null hypothesis: Assignement of browsers is statistically independent (not depend on) of treatment status.

Alternate hypothesis: Assignment of browsers is statistically dependent on treatment status.


In [4]:
# Contigency table from above is needed for chi_squared test

from scipy.stats import chi2_contingency

chi2_contingency(Cont)

(1.422936352290961, 0.7001672398372076, 3, array([[68.16, 75.84],
        [17.04, 18.96],
        [28.4 , 31.6 ],
        [28.4 , 31.6 ]]))

From above, the p-value, 0.7 is not statistically significant on most commonly used significance levels (0.001, 0.05 and 0.1). This implies that we can not reject the null (assignment of browsers is independent of treatment) based on the evidence. Similarly, the expected values produced by the distribution are not the same as the observed values.

Therefore we can go ahead and analyse how landing page affects engagement.

NULL HYPOTHESIS: The landing page used has no impact on time spent on page.

ALTERNATIVE HYPOTHESIS: The landing page used has an impact on time spent.


We can do this by comparing mean time spent for each group.

In [5]:

Control = df.loc[df['group'] == 'Control', ['time_on_page']]
Treatment = df.loc[df['group'] == 'Treatment', ['time_on_page']]

In [6]:
Control.describe()

,time_on_page
count,142.000000
mean,4.175772
std,5.378239
min,0.105017
25%,1.167706
50%,1.988447
75%,4.977858
max,29.575908


In [7]:
Treatment.describe()

,time_on_page
count,158.000000
mean,6.041728
std,6.330287
min,0.368440
25%,2.113369
50%,3.442268
75%,7.111216
max,30.474860


As shown by the mean above, the treatment group clearly spent more time on average onthe page compared to the control group( 1 min more). But is it significant enough to draw any conlusions.

In this case we will use a two sample t-test to compare the two means.
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html

In [8]:
from scipy.stats import ttest_ind

ttest_ind(Control, Treatment)

Ttest_indResult(statistic=array([-2.73548808]), pvalue=array([0.0066022]))

The p-values is below a 5% significance level (not big enough). Therefore, we conclude that given the NULL (the landing page has no impact in observed results) It would be very unlikely(0.6%) to see the observed results. Therefore the null must be wrong because we saw the results and we REJECT THE NULL.